# Variant System

> System for handling component variants and states

In [ ]:
#| default_exp core.variants

In [ ]:
#| hide
from nbdev.showdoc import *

In [ ]:
#| export
from typing import Dict, List, Optional, Any, Union
from dataclasses import dataclass, field
from enum import Enum
from cjm_fasthtml_daisyui.core.types import CSSContributor, CSSClasses

## Variant Definition

A variant represents a set of classes that can be applied based on conditions.

In [ ]:
#| export
class StyleType(str, Enum):
    """Common style modifiers across components.
    
    These were previously in modifiers.ipynb but are now part of the
    unified variant system.
    """
    OUTLINE = "outline"
    GHOST = "ghost"
    SOFT = "soft"
    DASH = "dash"
    BORDER = "border"
    LINK = "link"
    GLASS = "glass"

## Glass Effect Mixin

Mixin for components that support the glass morphism effect.

In [ ]:
#| export
@dataclass
class HasGlass(CSSContributor):
    """Mixin for components that support glass morphism effect.
    
    This mixin provides the glass effect styling for daisyUI components.
    The glass effect creates a frosted glass appearance.
    """
    
    # Glass effect property
    glass: bool = False
    
    def get_css_classes(self) -> CSSClasses:
        """Get glass effect CSS classes.
        
        Returns:
            List of CSS class strings for glass effect
        """
        if self.glass:
            return ["glass"]
        return []

In [ ]:
#| export
@dataclass
class Variant:
    """Represents a component variant.
    
    Variants allow components to have different appearances based on props.
    Inspired by CVA (Class Variance Authority) pattern.
    """
    name: str  # Variant name (e.g., 'intent', 'size')
    options: Dict[str, Union[str, List[str]]]  # Option -> classes mapping
    default: Optional[str] = None  # Default option
    
    def get_classes(
        self,
        value: Optional[str]  # TODO: Add description
    ) -> List[str]:  # TODO: Add return description
        """Get classes for a variant value."""
        if value is None:
            value = self.default
            
        if value is None or value not in self.options:
            return []
            
        classes = self.options[value]
        if isinstance(classes, str):
            return classes.split()
        return classes

## HasVariants Mixin

Mixin for components that use the variant system.

In [ ]:
#| export
class HasVariants(CSSContributor):
    """Mixin for components with variant support.
    
    This provides a more flexible alternative to individual boolean flags
    for component variations.
    """
    
    # Variant values (set by subclasses)
    variant_values: Dict[str, Optional[str]] = field(default_factory=dict)
    
    @classmethod
    def variants(
        cls  # TODO: Add type hint and description
    ) -> Dict[str, Variant]:  # TODO: Add return description
        """Define available variants.
        
        Subclasses should override this to define their variants.
        """
        return {}
    
    def get_css_classes(self) -> CSSClasses:
        """Get all classes from variants.
        
        Returns:
            List of CSS class strings from variants
        """
        classes = []
        variants = self.variants()
        
        for variant_name, variant in variants.items():
            value = self.variant_values.get(variant_name)
            classes.extend(variant.get_classes(value))
            
        return classes
    
    def set_variant(
        self,
        name: str,  # TODO: Add description
        value: str  # TODO: Add description
    ) -> 'HasVariants':  # TODO: Add return description
        """Set a variant value and return self for chaining."""
        self.variant_values[name] = value
        return self

## Compound Variants

Support for variants that depend on combinations of other variants.

In [ ]:
#| export
@dataclass
class CompoundVariant:
    """Represents a compound variant that depends on multiple conditions."""
    conditions: Dict[str, str]  # variant_name -> required_value
    classes: Union[str, List[str]]  # Classes to apply when conditions met
    
    def matches(
        self,
        variant_values: Dict[str, Optional[str]]  # TODO: Add description
    ) -> bool:  # TODO: Add return description
        """Check if all conditions are met."""
        for variant_name, required_value in self.conditions.items():
            if variant_values.get(variant_name) != required_value:
                return False
        return True
    
    def get_classes(
        self
    ) -> List[str]:  # TODO: Add return description
        """Get classes for this compound variant."""
        if isinstance(self.classes, str):
            return self.classes.split()
        return self.classes

In [ ]:
#| export
class HasCompoundVariants(HasVariants):
    """Extended mixin that supports compound variants."""
    
    @classmethod
    def compound_variants(
        cls  # TODO: Add type hint and description
    ) -> List[CompoundVariant]:  # TODO: Add return description
        """Define compound variants.
        
        Subclasses should override this to define compound variants.
        """
        return []
    
    def get_css_classes(self) -> CSSClasses:
        """Get all classes from variants including compound variants.
        
        Returns:
            List of CSS class strings from variants and compound variants
        """
        classes = super().get_css_classes()
        
        # Check compound variants
        for compound in self.compound_variants():
            if compound.matches(self.variant_values):
                classes.extend(compound.get_classes())
                
        return classes

## Predefined Variants

Common variants that can be reused across components.

In [ ]:
#| export
# Predefined style variant that can be used by components
STYLE_VARIANT = Variant(
    name="style",
    options={
        StyleType.OUTLINE.value: "outline",
        StyleType.GHOST.value: "ghost",
        StyleType.SOFT.value: "soft",
        StyleType.LINK.value: "link",
        StyleType.GLASS.value: "glass",
        StyleType.DASH.value: "dash",
        StyleType.BORDER.value: "border"
    },
    default=None
)

# Helper function to create style variant for a specific component
def create_style_variant(component_prefix: str) -> Variant:
    """Create a style variant with component-specific class names.
    
    Args:
        component_prefix: The component prefix (e.g., 'btn', 'badge')
        
    Returns:
        Variant with component-specific style classes
    """
    return Variant(
        name="style",
        options={
            style.value: f"{component_prefix}-{style.value}"
            for style in StyleType
        },
        default=None
    )

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()